In [1]:
import pandas as pd
from scipy import sparse 
import numpy as np

In [6]:
!curl -O http://files.grouplens.org/datasets/movielens/ml-100k.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 4808k    0  2241    0     0   1315      0  1:02:24  0:00:01  1:02:23  1315
  1 4808k    1 90441    0     0  33854      0  0:02:25  0:00:02  0:02:23 33860
  5 4808k    5  245k    0     0  69329      0  0:01:11  0:00:03  0:01:08 69344
  8 4808k    8  429k    0     0  98017      0  0:00:50  0:00:04  0:00:46 98028
 13 4808k   13  662k    0     0   123k      0  0:00:39  0:00:05  0:00:34  138k
 19 4808k   19  948k    0     0   148k      0  0:00:32  0:00:06  0:00:26  202k
 26 4808k   26 1269k    0     0   171k      0  0:00:27  0:00:07  0:00:20  250k
 35 4808k   35 1700k    0     0   202k      0  0:00:23  0:00:08  0:00:15  305k
 47 4808k   47 2284k    0     0   243k      0  0:00

'head' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!unzip ml-100k.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


[link1](https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/)
[link2](https://sdsawtelle.github.io/blog/output/week9-recommender-andrew-ng-machine-learning-with-python.html)

In [22]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=names)

In [23]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# cols = ['user_id','item_id', 'rating' , 'timestamp']


# df = pd.DataFrame({'user_id':[196,186,22,244,166],
#                   'item_id':[242,302,377,51,346],
#                   'rating':[3,3,1,2,1],
#                   'timestamp':[881250949,891717742,878887116,880606923,886397596]})

In [3]:
df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [20]:
df.columns

Index(['196', '242', '3', '881250949'], dtype='object')

In [24]:
n_users = df['user_id'].unique().shape[0]
n_items = df['item_id'].unique().shape[0]
print (str(n_users) + ' users')
print (str(n_items) + ' items')

943 users
1682 items


In [25]:
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [27]:
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print ('Sparsity: {:4.2f}%'.format(sparsity))

Sparsity: 6.30%


In [32]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [33]:
train, test = train_test_split(ratings)

Using cosine sim

In [34]:
def slow_similarity(ratings, kind='user'):
    if kind == 'user':
        axmax = 0
        axmin = 1
    elif kind == 'item':
        axmax = 1
        axmin = 0
    sim = np.zeros((ratings.shape[axmax], ratings.shape[axmax]))
    for u in range(ratings.shape[axmax]):
        for uprime in range(ratings.shape[axmax]):
            rui_sqrd = 0.
            ruprimei_sqrd = 0.
            for i in range(ratings.shape[axmin]):
                sim[u, uprime] = ratings[u, i] * ratings[uprime, i]
                rui_sqrd += ratings[u, i] ** 2
                ruprimei_sqrd += ratings[uprime, i] ** 2
            sim[u, uprime] /= rui_sqrd * ruprimei_sqrd
    return sim

In [35]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [38]:
# %timeit 
# slow_similarity(train)

In [42]:
user_similarity = fast_similarity(train, kind='user')
item_similarity = fast_similarity(train, kind='item')
print (item_similarity[:4, :4])

[[1.         0.39403072 0.30424369 0.45753293]
 [0.39403072 1.         0.23696246 0.48676469]
 [0.30424369 0.23696246 1.         0.27606481]
 [0.45753293 0.48676469 0.27606481 1.        ]]


In [44]:
item_similarity.shape

(1682, 1682)

In [47]:
user_similarity.shape

(943, 943)

In [54]:
def predict_slow_simple(ratings, similarity, kind='user'):
    pred = np.zeros(ratings.shape)
    if kind == 'user':
        for i in range(ratings.shape[0]):
            for j in range(ratings.shape[1]):
                pred[i, j] = similarity[i, :].dot(ratings[:, j])\
                             /np.sum(np.abs(similarity[i, :]))
        return pred
    elif kind == 'item':
        for i in range(ratings.shape[0]):
            for j in range(ratings.shape[1]):
                pred[i, j] = similarity[j, :].dot(ratings[i, :].T)\
                             /np.sum(np.abs(similarity[j, :]))

        return pred

def predict_fast_simple(ratings, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'item':
        return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [ ]:
%timeit predict_slow_simple(train, user_similarity, kind='user')

In [56]:
%timeit predict_fast_simple(train, user_similarity, kind='user')

86 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Test 

In [57]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [58]:
item_prediction = predict_fast_simple(train, item_similarity, kind='item')
user_prediction = predict_fast_simple(train, user_similarity, kind='user')

print ('User-based CF MSE: ' + str(get_mse(user_prediction, test)))
print ('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))

User-based CF MSE: 8.468660018717888
Item-based CF MSE: 11.546083763964022


In [95]:
5/25

0.2

In [96]:
# Split the dataframe into a train and test set
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df,test_size=0.25)


In [97]:
train_data = pd.DataFrame(train_data)

test_data = pd.DataFrame(test_data)


In [98]:
train_data

,user_id,item_id,rating,timestamp
2,22,377,1,878887116
1,186,302,3,891717742
0,196,242,3,881250949


In [99]:
test_data

,user_id,item_id,rating,timestamp
3,244,51,2,880606923
4,166,346,1,886397596


In [100]:
# Create training and test matrix


R = np.zeros((n_u,n_m))

In [101]:
R

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [102]:
for i in train_data.itertuples():
    print(i)

Pandas(Index=2, user_id=22, item_id=377, rating=1, timestamp=878887116)
Pandas(Index=1, user_id=186, item_id=302, rating=3, timestamp=891717742)
Pandas(Index=0, user_id=196, item_id=242, rating=3, timestamp=881250949)


In [90]:
E=[]

In [91]:
E[train_data.itertuples()[1]-1, train_data.itertuples()[1]-1] =train_data.itertuples() 

TypeError: 'map' object is not subscriptable

In [118]:
R = np.zeros((n_u, n_m))
for line in train_data.itertuples():
    R[line[0]-1, line[1]-1] = line[2]  
    
# T = np.zeros((n_u, n_m))
# for line in test_data.itertuples():
#     T[line[1]-1, line[2]-1] = line[3]

IndexError: index 21 is out of bounds for axis 1 with size 5

In [110]:
R = train_data.pivot_table(index='user_id', columns="item_id", values= 'rating',fill_value=0 )

In [114]:
np.array(R)

array([[0, 0, 1],
       [0, 3, 0],
       [3, 0, 0]], dtype=int64)

In [111]:
T = test_data.pivot_table(index='user_id', columns="item_id", values= 'rating',fill_value=0 )

In [116]:
n_m

5

In [40]:
# making pivot tabel
pivot = df.pivot_table(columns = 'item_id', 
                       index ='user_id', 
                       values ='rating', 
                       fill_value=0)

In [41]:
pivot

item_id,51,242,302,346,377
user_id,,,,,
22,3,0,0,0,1
166,0,0,0,1,0
186,0,1,3,0,0
196,0,3,0,5,0
244,2,0,0,0,0


In [43]:
spa= sparse.csr_matrix(pivot)

In [49]:
spa.dtype

dtype('int64')